# Possible Application Whitelisting Bypass via dll loaded by odbcconf.exe
Detects defence evasion attempt via odbcconf.exe execution to load DLL

## Rule Content
```
- title: Possible Application Whitelisting Bypass via dll loaded by odbcconf.exe
  id: 65d2be45-8600-4042-b4c0-577a1ff8a60e
  description: Detects defence evasion attempt via odbcconf.exe execution to load
    DLL
  status: experimental
  references:
  - https://github.com/LOLBAS-Project/LOLBAS/blob/master/yml/OSBinaries/Odbcconf.yml
  - https://twitter.com/Hexacorn/status/1187143326673330176
  author: Kirill Kiryanov, Beyu Denis, Daniil Yugoslavskiy, oscd.community
  date: 2019/10/25
  modified: 2019/11/07
  tags:
  - attack.defense_evasion
  - attack.execution
  - attack.t1218
  logsource:
    category: process_creation
    product: windows
    service: null
  detection:
    selection_1:
      Image|endswith: \odbcconf.exe
      CommandLine|contains:
      - -f
      - regsvr
    selection_2:
      ParentImage|endswith: \odbcconf.exe
      Image|endswith: \rundll32.exe
    condition: selection_1 or selection_2
  level: medium
  falsepositives:
  - Legitimate use of odbcconf.exe by legitimate user

```

## Querying Elasticsearch

### Import Libraries

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import pandas as pd

### Initialize Elasticsearch client

In [ ]:
es = Elasticsearch(['http://helk-elasticsearch:9200'])
searchContext = Search(using=es, index='logs-*', doc_type='doc')

### Run Elasticsearch Query

In [ ]:
s = searchContext.query('query_string', query='((process_path.keyword:*\\odbcconf.exe AND process_command_line.keyword:(*\-f* OR *regsvr*)) OR (process_parent_path.keyword:*\\odbcconf.exe AND process_path.keyword:*\\rundll32.exe))')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

### Show Results

In [ ]:
df.head()